In [ ]:
import sys
!cp ../input/rapids/rapids.0.16.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import cupy as cp
import cudf 
from sklearn.model_selection import StratifiedKFold
from cuml.metrics import roc_auc_score
from cuml import svm, neighbors, linear_model
from sklearn.preprocessing import StandardScaler
from cuml.decomposition import PCA
from tqdm import tqdm

In [ ]:
train = cudf.read_csv('../input/instant-gratification/train.csv')
test = cudf.read_csv('../input/instant-gratification/test.csv')

In [ ]:
oof_svc = cp.zeros(len(train)) 
pred_te_svc = cp.zeros(len(test))

oof_knn = cp.zeros(len(train)) 
pred_te_knn = cp.zeros(len(test))

oof_lr = cp.zeros(len(train)) 
pred_te_lr = cp.zeros(len(test))

In [ ]:
cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [ ]:
for i in tqdm(range(512)):
    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)
    
    data = cudf.concat([cudf.DataFrame(train2[cols]), cudf.DataFrame(test2[cols])])
    data2 = cp.array(StandardScaler().fit_transform(PCA(svd_solver='full',n_components=100).fit_transform(data[cols].as_gpu_matrix())))
    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    for train_index, test_index in skf.split(train2[cols].values, train2['target'].to_array()):
        clf = linear_model.LogisticRegression(C=0.1)
        clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        oof_lr[idx1[test_index].to_array()] = clf.predict_proba(train3[test_index,:])[:,1]
        if idx2.shape[0] != 0:
            pred_te_lr[idx2.to_array()] += clf.predict_proba(test3)[:,1] / skf.n_splits
            
        clf = svm.SVC(probability=True, kernel='poly', degree=4, gamma='auto')
        clf.fit(cp.asnumpy(train3[train_index,:]),cp.asnumpy(train2.loc[train_index]['target']))
        oof_svc[idx1[test_index].to_array()] = clf.predict_proba(train3[test_index,:])[:,1]
        if idx2.shape[0] != 0:
            pred_te_svc[idx2.to_array()] += clf.predict_proba(test3)[:,1] / skf.n_splits
            
        clf = neighbors.KNeighborsClassifier(n_neighbors=17, p=2.9)
        clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        oof_knn[idx1[test_index].to_array()] = clf.predict_proba(train3[test_index,:])[:,1]
        if idx2.shape[0] != 0:
            pred_te_knn[idx2.to_array()] += clf.predict_proba(test3)[:,1] / skf.n_splits

In [ ]:
print('knn', roc_auc_score(train['target'], oof_knn))

In [ ]:
print('lr', roc_auc_score(train['target'], oof_lr))

In [ ]:
print('svc', roc_auc_score(train['target'], oof_svc))

In [ ]:
print('blend', roc_auc_score(train['target'], (oof_svc+oof_knn+oof_lr)/3))

In [ ]:
sub = cudf.read_csv('../input/instant-gratification/sample_submission.csv')

In [ ]:
sub['target'] = (pred_te_svc + pred_te_knn + pred_te_lr)/3
sub.to_csv('submission.csv', index=False)